In [1]:
import pandas as pd

from tqdm import tqdm
from copy import deepcopy

In [2]:
dataset_train = pd.read_parquet("dataset_train_small.parquet")
target_train = pd.read_parquet("target_train_small.parquet")

dataset_test = pd.read_parquet("dataset_test.parquet")
target_test = pd.read_parquet("target_test_contest.parquet")

In [3]:
num_cat_features = [
    'amnt', 'currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group','ecommerce_flag',
    'payment_system', 'mcc', 'country', 'city', 'mcc_category', 'day_of_week', 'hour', 'days_before', 'weekofyear', 'hour_diff'
]

cat_features = [
    'currency', 'operation_kind', 'card_type', 'operation_type', 'operation_type_group','ecommerce_flag',
    'payment_system', 'mcc', 'country', 'city', 'mcc_category', 'day_of_week', 'hour', 'days_before', 'weekofyear', 'hour_diff'
]

aggs = ['mean', 'max', 'min', 'count', 'std', 'median', 'sum', set]

In [4]:
train_groupped_by_income_flag = dataset_train.groupby(['app_id', 'income_flag'])[num_cat_features].agg(aggs).reset_index()
test_groupped_by_income_flag = dataset_test.groupby(['app_id', 'income_flag'])[num_cat_features].agg(aggs).reset_index()

train_groupped_by_income_flag = train_groupped_by_income_flag.sort_values('app_id')
test_groupped_by_income_flag = test_groupped_by_income_flag.sort_values('app_id')

In [5]:
base_data = {}
    
for i in range(1, 4):
    for col in train_groupped_by_income_flag.columns:
        if 'app_id' not in col and 'income_flag' not in col:
            base_data[f'{col}_{i}'] = 0

In [ ]:
new_train_dataset = []

last_app_id = -1           
    
for _, row in zip(tqdm(range(len(train_groupped_by_income_flag))), train_groupped_by_income_flag.iterrows()):
    row = row[1]
    app_id = row['app_id'][0]
    
    if app_id != last_app_id:
        if last_app_id != -1:
            new_train_dataset.append(data)
        data = {**{'app_id': app_id} , **deepcopy(base_data)}
        last_app_id = app_id

    for col in row.items():
        col_name = col[0]
        col_value = col[1]

        if 'app_id' not in col_name and 'income_flag' not in col_name:
            if 'set' in col_name:
                col_value = len(col_value)

            data[f'{col_name}_{int(row["income_flag"])}'] = col_value
    
new_train_dataset = pd.DataFrame(new_train_dataset)

In [ ]:
new_test_dataset = []

last_app_id = -1           
    
for _, row in zip(tqdm(range(len(test_groupped_by_income_flag))), test_groupped_by_income_flag.iterrows()):
    row = row[1]
    app_id = row['app_id'][0]
    
    if app_id != last_app_id:
        if last_app_id != -1:
            new_test_dataset.append(data)
        data = {**{'app_id': app_id} , **deepcopy(base_data)}
        last_app_id = app_id

    for col in row.items():
        col_name = col[0]
        col_value = col[1]

        if 'app_id' not in col_name and 'income_flag' not in col_name:
            if 'set' in col_name:
                col_value = len(col_value)

            data[f'{col_name}_{int(row["income_flag"])}'] = col_value
    
new_test_dataset = pd.DataFrame(new_test_dataset)

In [ ]:
cat_features_for_catboost = [feature for feature in new_train_dataset.columns if any([f in feature for f in cat_features]) and any([agg in feature.split(',')[1] for agg in ['max', 'min', 'count', 'set']])] + ['product']

In [ ]:
new_train_dataset['product'] = target_train['product'].tolist()
new_train_dataset['flag'] = target_train['flag'].tolist()

In [ ]:
new_test_dataset['product'] = target_test['product'].tolist()

In [ ]:
for f in cat_features_for_catboost:
    new_train_dataset[f] = new_train_dataset[f].astype(np.int32)

In [ ]:
for f in cat_features_for_catboost:
    new_test_dataset[f] = new_test_dataset[f].astype(np.int32)

In [ ]:
new_train_dataset = new_train_dataset.fillna(0)
new_test_dataset = new_test_dataset.fillna(0)

In [ ]:
new_train_dataset.to_csv('new_train_dataset.csv')
new_test_dataset.to_csv('new_test_dataset.csv')